### 0. Installing Dependencies for YOLOv6

In [ ]:
!pip install -r /notebooks/requirements.txt
!pip install onnxruntime pillow configparser opencv-python gradio roboflow tensorboard
!wget https://huggingface.co/AICademyHK/YOLOv6s/resolve/main/yolov6s.onnx -O /notebooks/yolov6s.onnx
!mkdir /notebooks/weights
!wget https://github.com/meituan/YOLOv6/releases/download/0.3.0/yolov6s.pt -O /notebooks/weights/yolov6s.pt

### 1. Downloading the dataset, renaming the folders for YOLOv6 Training

In [ ]:
!cd /notebooks

from roboflow import Roboflow
rf = Roboflow(api_key="Uyxx7cbmYWcqWg74unuj")
project = rf.workspace("joseph-nelson").project("chess-pieces-new")
version = project.version(24)
dataset = version.download("mt-yolov6")


!mv /notebooks/Chess-Pieces-24/images/valid /notebooks/Chess-Pieces-24/images/val
!mv /notebooks/Chess-Pieces-24/labels/valid /notebooks/Chess-Pieces-24/labels/val

### 2. Setup the dataset.yaml

--COPY and PASTE the following into /notebooks/data/dataset.yaml --

```
# Please ensure that your custom_dataset are put in same parent dir with YOLOv6_DIR
train: /notebooks/Chess-Pieces-24/images/train # train images
val: /notebooks/Chess-Pieces-24/images/val # val images
test: /notebooks/Chess-Pieces-24/images/test # test images (optional)

# whether it is coco dataset, only coco dataset should be set to True.
is_coco: False
# Classes
nc: 13  # number of classes
names: [
    "bishop",
    "black-bishop",
    "black-king",
    "black-knight",
    "black-pawn",
    "black-queen",
    "black-rook",
    "white-bishop",
    "white-king",
    "white-knight",
    "white-pawn",
    "white-queen",
    "white-rook"
]  # class names
```

### 3. Readying for the training

Deciding on the parameters
Key Parameters to Tune:
- `batch_size`: Number of images the model sees before updating the learning parameters.
- `epochs`: How many times the model will go through the entire dataset.
- `img_size`: Size to which all the images are resized.


The following command trains the chess dataset on a yolov6n nano model

In [ ]:
!python tools/train.py --batch 16 --epochs 50 --img-size 416 --data data/dataset.yaml --device 0 --name tutorial

### 4. Evaluate the model 

In [ ]:
!python tools/eval.py --data data/dataset.yaml --batch 16 --weights /notebooks/runs/train/tutorial/weights/best_ckpt.pt --img-size 416 --task val --do_pr_metric True --plot_confusion_matrix

### 5. Checking the performance of the model